<a href="https://colab.research.google.com/github/binliu0630/transformers/blob/master/Transformers_multilabel_Classification_Fine_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Mon Jul 19 18:29:35 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
%%capture
!pip install transformers datasets pandas torch kaggle


# Imports 

In [3]:
import transformers
import datasets
print(f"transformers version: {transformers.__version__} and dataset version: {datasets.__version__}")

transformers version: 4.8.2 and dataset version: 1.9.0


In [35]:
import torch
import pandas as pd
import numpy as np
from pathlib import Path
from datasets import load_dataset
from transformers import (AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer)

# Load Data

In [15]:
!mkdir .kaggle

mkdir: cannot create directory ‘.kaggle’: File exists


In [16]:
import json
token = {"username":"binliu0630","key":"8fbf745edbb9799f7f8f45f516701290"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

In [21]:
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json

In [22]:
!kaggle config set -n path -v{/content}

- path is now set to: {/content}


In [23]:
!chmod 600 /root/.kaggle/kaggle.json

In [24]:
!kaggle competitions download -c jigsaw-toxic-comment-classification-challenge -p /content

  0% 0.00/1.39M [00:00<?, ?B/s]
100% 1.39M/1.39M [00:00<00:00, 85.3MB/s]
 65% 17.0M/26.3M [00:00<00:00, 175MB/s]
100% 26.3M/26.3M [00:00<00:00, 169MB/s]
 21% 5.00M/23.4M [00:00<00:00, 42.8MB/s]
100% 23.4M/23.4M [00:00<00:00, 114MB/s] 
  0% 0.00/1.46M [00:00<?, ?B/s]
100% 1.46M/1.46M [00:00<00:00, 208MB/s]


In [25]:
!unzip \*.zip

Archive:  sample_submission.csv.zip
  inflating: sample_submission.csv   

Archive:  test_labels.csv.zip
  inflating: test_labels.csv         

Archive:  train.csv.zip
  inflating: train.csv               

Archive:  test.csv.zip
  inflating: test.csv                

4 archives were successfully processed.


In [26]:
data = pd.read_csv('train.csv')
data.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [27]:
data.shape

(159571, 8)

# Preprocess Data

In [30]:
# create labels column
label_cols = [c for c in data.columns if c not in ['id', 'comment_text']]
label_cols


['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [31]:
data['labels'] = data[label_cols].values.tolist()

In [32]:
data.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,labels
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0]"
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0]"
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0]"
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0]"
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0]"


In [33]:
# small sample for quick prototyping
data_sample = data.sample(1000)
data_sample.shape


(1000, 9)

In [38]:
# create train/test split - 
mask = np.random.rand(len(data_sample))<0.8
data_train = data_sample[mask]
data_test = data_sample[~mask]
data_train.shape, data_test.shape

In [40]:
# create train/test split - another approach
from sklearn.model_selection import train_test_split
data_train, data_test = train_test_split(data_sample, test_size=0.2)
data_train.shape, data_test.shape

((800, 9), (200, 9))

# Tokenize and Encode

In [41]:
checkpoint = 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [65]:
train_labels = data_train['labels'].values.tolist()
test_labels = data_test['labels'].values.tolist()

In [64]:
train_labels


[[0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [1, 1, 1, 0, 1, 0],
 [0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [1, 0, 1, 0, 1, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0,

In [66]:
train_encoding = tokenizer(data_train['comment_text'].values.tolist(), truncation=True)
test_encoding = tokenizer(data_test['comment_text'].values.tolist(), truncation=True)

In [84]:
class JigsawDataset(torch.utils.data.Dataset):
  def __init__(self, encodings, labels):
    self.encodings = encodings
    self.labels = labels
  
  def __getitem__(self, idx):
    item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    item['labels'] = torch.tensor([int(x) for x in self.labels[idx]]).to(torch.float)
    return item
  
  def __len__(self):
    return len(self.labels)


In [85]:
train_dataset = JigsawDataset(train_encoding, train_labels)
test_dataset = JigsawDataset(test_encoding, test_labels)

In [86]:
train_dataset[10]

{'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1]),
 'input_ids': tensor([ 101, 2053, 2009, 1005, 1055, 2025, 1012, 2045, 2024, 2145, 2195, 5939,
         6873, 2015, 1010, 1998, 1996, 3452, 3015, 2071, 2022, 2488, 1012,  100,
         1006, 2831, 1007,  102]),
 'labels': tensor([0., 0., 0., 0., 0., 0.])}

In [87]:
tokenizer.decode(train_dataset[0]['input_ids'])

'[CLS] dont worry sister we are now going to adopt a different policy we will search who is this mezo mezo through legal actions. and we will soon find out who is this guy? and then we will take a legal action i have done mail to relative authorities and u will soon see this satanic people coming to light. [SEP]'

# Load Model

In [88]:
num_labels = len(label_cols)

In [89]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=num_labels, problem_type='multi_label_classification').to('cuda')

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.d423bdf2f58dc8b77d5f5d18028d7ae4a72dcfd8f468e81fe979ada957a8c361
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "problem_type": "multi_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoida

# Load Trainer

In [100]:
# config logging steps to see train loss
batch_size = 8
logging_steps = len(train_dataset)//batch_size

In [101]:
args = TrainingArguments('.', 
                         evaluation_strategy = 'epoch',
                         logging_steps=logging_steps,
                         num_train_epochs=3)
trainer = Trainer(model=model, args=args, train_dataset=train_dataset, eval_dataset=test_dataset, tokenizer=tokenizer)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [102]:
trainer.train()

***** Running training *****
  Num examples = 800
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 300


Epoch,Training Loss,Validation Loss
1,0.025200,0.074504
2,0.021200,0.066870
3,0.016600,0.067096


***** Running Evaluation *****
  Num examples = 200
  Batch size = 8
***** Running Evaluation *****
  Num examples = 200
  Batch size = 8
***** Running Evaluation *****
  Num examples = 200
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=300, training_loss=0.0210055947303772, metrics={'train_runtime': 62.8247, 'train_samples_per_second': 38.202, 'train_steps_per_second': 4.775, 'total_flos': 237185895870144.0, 'train_loss': 0.0210055947303772, 'epoch': 3.0})